In [71]:
#making widgets global

import ipywidgets as widgets
from IPython.display import display

prolific_id = widgets.Text(description="Prolific ID") #CORRECTED
user_plant = widgets.RadioButtons(description="Top Plant:") #CORRECTED
user_capacity = widgets.RadioButtons(description="Capacity:") #CORRECTED
user_answer = widgets.Text(description="Solar Plants") #COORECTED
user_answer = widgets.BoundedFloatText(description="% LA", min=0, max=100) #CORRECTED
user_answer = widgets.FloatText(description="MW Difference") #corrected
user_answer = widgets.Text(description="Hydro in NorCal") #CORRECTED
user_answer = widgets.Text(description="Operator") #not working for whatever reason
user_answer = widgets.Text(description="Energy Source") #CORRECTED
user_answer = widgets.Text(description="Agreement") #CORRECTED

# Display them for debugging
display(
    prolific_id_input, plant_question, capacity_question, solar_input, percentage_input,difference_input, hydro_input, answer_input, source_input, likert_scale)#, operator_input  likert_scale

Text(value='dahsd', description='Prolific ID:', layout=Layout(width='50%'), placeholder='Enter your Prolific I…

RadioButtons(description='Top Plant:', index=2, layout=Layout(width='400px'), options=('Castaic (Pumped Storag…

RadioButtons(description='Capacity:', index=1, layout=Layout(width='300px'), options=('1200', '5664', '1800', …

Text(value='2', description='Answer:')

BoundedFloatText(value=5.0, description='% in LA:', step=0.1, style=DescriptionStyle(description_width='initia…

BoundedFloatText(value=150.0, description='MW Difference:', max=10000.0, min=-10000.0, step=10.0, style=Descri…

Text(value='22', description='Answer:', placeholder='e.g. 12 hydro plants found across 5 counties')

Text(value='SCE', description='Answer:', placeholder='e.g. Calpine', style=TextStyle(description_width='initia…

RadioButtons(description='Your Answer:', index=1, layout=Layout(width='500px'), options=('OGW', 'SUN', 'OBG', …

RadioButtons(description='Your Answer:', index=2, layout=Layout(width='500px'), options=('1 - Strongly Disagre…

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊To start: Enter your Prolific ID.</h2>
</div>


In [96]:
import ipywidgets as widgets
from IPython.display import display

# Text input
prolific_id_input = widgets.Text(
    placeholder='Enter your Prolific ID',
    description='Prolific ID:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Submit button
submit_button = widgets.Button(description="Submit")
output = widgets.Output()

def on_submit_clicked(b):
    with output:
        output.clear_output()
        prolific_id = prolific_id_input.value
        print(f"✅ Captured Prolific ID: {prolific_id}")

submit_button.on_click(on_submit_clicked)

# Display the widget
display(prolific_id_input, submit_button, output)


Text(value='', description='Prolific ID:', layout=Layout(width='50%'), placeholder='Enter your Prolific ID', s…

Button(description='Submit', style=ButtonStyle())

Output()

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 1) What is the name of power plant with the highest generation capacity? What is its Generation Capacity?</h2>
</div>


In [2]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
import random
import time
from IPython.display import display, clear_output

def start_capacity_quiz():
    # Declare global widget variables so they're accessible later
    global plant_question, capacity_question, correct_plant, correct_capacity, start_time

    # Load and clean dataset
    df = pd.read_csv("Power_Plant.csv")
    df['Capacity_Latest'] = pd.to_numeric(df['Capacity_Latest'], errors='coerce')
    df = df.dropna(subset=['Capacity_Latest'])

    # Get top 20 plants by capacity
    top_capacity = df.sort_values(by='Capacity_Latest', ascending=False).head(20)

    # Create Altair chart
    chart = alt.Chart(top_capacity).mark_bar().encode(
        x=alt.X('Capacity_Latest:Q', title='Capacity (MW)'),
        y=alt.Y('PlantName:N', sort='-x', title='Power Plant'),
        tooltip=['PlantName', 'Capacity_Latest', 'Operator Company', 'County', 'x', 'y']
    ).properties(
        title='Top 20 Power Plants by Generation Capacity',
        height=600
    )

    display(chart)

    # Correct answers
    correct_plant = top_capacity.iloc[0]['PlantName']
    correct_capacity = str(int(top_capacity.iloc[0]['Capacity_Latest']))

    # Randomized options (5 each)
    plant_pool = list(top_capacity['PlantName'].unique())
    plant_choices = random.sample([p for p in plant_pool if p != correct_plant], 4) + [correct_plant]
    random.shuffle(plant_choices)

    capacity_pool = list(top_capacity['Capacity_Latest'].dropna().astype(int).astype(str).unique())
    capacity_choices = random.sample([c for c in capacity_pool if c != correct_capacity], 4) + [correct_capacity]
    random.shuffle(capacity_choices)

    # Widgets now stored as global variables
    plant_question = widgets.RadioButtons(
        options=plant_choices,
        description='Top Plant:',
        layout=widgets.Layout(width='400px')
    )

    capacity_question = widgets.RadioButtons(
        options=capacity_choices,
        description='Capacity:',
        layout=widgets.Layout(width='300px')
    )

    submit_button = widgets.Button(description="Submit Answer")
    submission_output = widgets.Output()

    if 'capacity_quiz_responses' not in globals():
        global capacity_quiz_responses
        capacity_quiz_responses = []

    start_time = time.time()

    def on_submit(b):
        elapsed = time.time() - start_time
        user_plant = plant_question.value
        user_capacity = capacity_question.value

        capacity_quiz_responses.append({
            'user_plant': user_plant,
            'user_capacity': user_capacity,
            'correct_plant': correct_plant,
            'correct_capacity': correct_capacity,
            'is_correct': (user_plant == correct_plant and user_capacity == correct_capacity),
            'time_sec': round(elapsed, 2)
        })

        with submission_output:
            clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded.</b> Time taken: {round(elapsed, 2)} seconds.<br>"
                f"You may proceed to the next task."
            ))

    submit_button.on_click(on_submit)

    display(widgets.VBox([
        widgets.HTML("<h4>📊 Questions: Based on the chart above...</h4>"),
        plant_question,
        capacity_question,
        submit_button,
        submission_output
    ]))
start_capacity_quiz()

alt.Chart(...)

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 2) How many solar plants are located in California?</h2>
</div>


In [16]:
def start_solar_task():
    global energy_filter, response_box, solar_input
    
    df = pd.read_csv("Power_Plant.csv")
    df['Capacity_Latest'] = pd.to_numeric(df['Capacity_Latest'], errors='coerce')
    df = df.dropna(subset=['Capacity_Latest'])

    clear_output()
    print("☀️ Task: How many solar plants are located in CA?")
    print("Use the dropdown to filter the chart, then submit your answer. The code for solar plants is SUN.")

    start_time = time.time()

    energy_filter = widgets.Dropdown(
        options=sorted(df['PriEnergySource'].dropna().unique()),
        value='AB',
        description='Energy Source:'
    )

    chart_output = widgets.Output()
    submission_output = widgets.Output()
    solar_input = widgets.Text(description='Answer:')  # ← global response box for this task

    def update_chart(change=None):
        with chart_output:
            clear_output()
            selected_fuel = energy_filter.value
            filtered = df[df['PriEnergySource'] == selected_fuel]
            count = len(filtered)

            chart = alt.Chart(pd.DataFrame({'count': [count]})).mark_text(
                align='center',
                baseline='middle',
                fontSize=60
            ).encode(
                text='count:Q'
            ).properties(
                width=200,
                height=120,
                title=f"Total {selected_fuel.title()} Plants in CA"
            )
            display(chart)

    energy_filter.observe(update_chart, names='value')
    update_chart()

    submit_button = widgets.Button(description="Submit Answer")

    def on_submit(b):
        elapsed = time.time() - start_time
        responses.append({
            'question': 'Q: How many solar plants are in CA?',
            'user_answer': solar_input.value,
            'correct_answer': str(len(df[df['PriEnergySource'] == 'SUN'])),
            'selected_filter': energy_filter.value,
            'time_sec': round(elapsed, 2)
        })
        with submission_output:
            clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded.</b> Time taken: {round(elapsed, 2)} seconds.<br>"
                f"You may proceed to the next task."
            ))

    submit_button.on_click(on_submit)

    display(widgets.VBox([
        energy_filter,
        chart_output,
        solar_input,
        submit_button,
        submission_output
    ]))


# Launch the solar task
start_solar_task()


☀️ Task: How many solar plants are located in CA?
Use the dropdown to filter the chart, then submit your answer. The code for solar plants is SUN.


<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 3) What percentage of solar plants are in Los Angeles County?</h2>
</div>


In [17]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
import time
from IPython.display import display, clear_output

def start_solar_pct_task():
    global percentage_input
    clear_output()
    print("☀️ Task: There are 925 total solar plants in California. What percentage of solar plants are located in Los Angeles County?")

    # Ensure global responses list exists
    if 'responses' not in globals():
        global responses
        responses = []

    start_time = time.time()

    # Filter solar plants
    solar_df = df[df['PriEnergySource'] == 'SUN']
    county_counts = solar_df['County'].str.upper().value_counts().reset_index()
    county_counts.columns = ['County', 'Plant Count']

    # Calculate correct answer
    la_count = county_counts[county_counts['County'] == 'LOS ANGELES']['Plant Count'].values[0]
    total_count = county_counts['Plant Count'].sum()
    percent_actual = round((la_count / total_count) * 100, 2)

    # Display chart
    chart = alt.Chart(county_counts).mark_bar().encode(
        y=alt.Y('County:N', sort='-x'),
        x=alt.X('Plant Count:Q'),
        tooltip=['County', 'Plant Count']
    ).properties(
        title='Solar Plants by County (Top Contributors)',
        width=600,
        height=400
    ).interactive()

    display(chart)

    # User input
    percentage_input = widgets.BoundedFloatText(
        min=0, max=100, step=0.1,
        description='% in LA:',
        style={'description_width': 'initial'}
    )

    submit_button = widgets.Button(description="Submit Answer")
    output = widgets.Output()

    def on_submit(b):
        elapsed = time.time() - start_time

        responses.append({
            'question': 'Q: What % of solar plants are in LA County?',
            'user_answer': percentage_input.value,
            'correct_answer': percent_actual,
            'time_sec': round(elapsed, 2)
        })

        with output:
            clear_output()
            display(widgets.HTML("✅ Answer recorded. You may proceed."))

    submit_button.on_click(on_submit)
    display(widgets.VBox([percentage_input, submit_button, output]))


# 🚀 Run the task
start_solar_pct_task()



☀️ Task: There are 925 total solar plants in California. What percentage of solar plants are located in Los Angeles County?


alt.Chart(...)

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 4) Enter the difference in MW capacity between solar and natural gas plants. </h2>
</div>

In [19]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
import time
from IPython.display import display, clear_output

def start_capacity_comparison_task():
    global difference_input
    clear_output()
    print("⚡ Task: Estimate the MW difference in total capacity between solar and natural gas plants.")
    print("Use the chart below and enter your best guess for: (solar MW) - (natural gas MW)")

    # Aggregate capacity by primary energy source
    df['Capacity_Latest'] = pd.to_numeric(df['Capacity_Latest'], errors='coerce')
    source_totals = df.groupby('PriEnergySource')['Capacity_Latest'].sum().reset_index()
    filtered_sources = source_totals[source_totals['PriEnergySource'].isin(['SUN', 'NG'])]

    # Display chart
    chart = alt.Chart(filtered_sources).mark_bar().encode(
        x=alt.X('PriEnergySource:N', title='Energy Source'),
        y=alt.Y('Capacity_Latest:Q', title='Total Capacity (MW)'),
        tooltip=['PriEnergySource', 'Capacity_Latest']
    ).properties(
        title='Total Generation Capacity by Source (SUN vs NG)',
        width=400,
        height=300
    )
    display(chart)

    # Input and submission
    difference_input = widgets.BoundedFloatText(
        value=0,
        min=-10000,
        max=10000,
        step=10,
        description='MW Difference:',
        style={'description_width': 'initial'}
    )

    submit_button = widgets.Button(description="Submit Answer")
    submission_output = widgets.Output()

    def on_submit(b):
        user_guess = difference_input.value
        actual_diff = filtered_sources.loc[filtered_sources['PriEnergySource'] == 'SUN', 'Capacity_Latest'].values[0] - \
                      filtered_sources.loc[filtered_sources['PriEnergySource'] == 'NG', 'Capacity_Latest'].values[0]

        responses.append({
            'question': 'Q: What is the MW capacity difference between solar and natural gas?',
            'user_answer': user_guess,
            'actual_difference': round(actual_diff),
            'is_close': abs(user_guess - actual_diff) < 500  # Optional threshold
        })

        with submission_output:
            clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded.</b><br>"
                #f"Actual Difference: {round(actual_diff)} MW<br>"
                f"You may proceed to the next task."
            ))

    submit_button.on_click(on_submit)
    display(widgets.VBox([difference_input, submit_button, submission_output]))


# 🚀 Launch the task
start_capacity_comparison_task()


⚡ Task: Estimate the MW difference in total capacity between solar and natural gas plants.
Use the chart below and enter your best guess for: (solar MW) - (natural gas MW)


alt.Chart(...)

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 5) How many hydroelectric plants are in Northern California? </h2>
</div>

In [22]:
#completed for survey; submit button worked
import pandas as pd
import altair as alt
import ipywidgets as widgets
import time
from IPython.display import display, clear_output

# Load and clean dataset
df = pd.read_csv("Power_Plant.csv")
df['PriEnergySource'] = df['PriEnergySource'].str.upper()
df['Capacity_Latest'] = pd.to_numeric(df['Capacity_Latest'], errors='coerce')

# Set up responses log if not already
if 'responses' not in globals():
    responses = []

def start_hydro_filter_task():
    global hydro_input  # ⬅️ Make the input globally accessible

    clear_output()
    start_time = time.time()

    # Get all unique energy sources
    energy_options = sorted(df['PriEnergySource'].dropna().unique().tolist())
    default_selection = ('AB',) if 'AB' in energy_options else (energy_options[0],)

    # Widgets
    energy_selector = widgets.SelectMultiple(
        options=energy_options,
        value=default_selection,
        description='Sources:',
        layout=widgets.Layout(width='250px', height='150px')
    )

    north_toggle = widgets.Checkbox(
        value=False,
        description='Show only Northern CA'
    )

    hint = widgets.HTML(value="<i>Hint: Try selecting the code WAT to find hydroelectric plants.</i>")

    chart_output = widgets.Output()
    count_output = widgets.Output()
    submission_output = widgets.Output()

    # Chart update function
    def update_chart(change=None):
        with chart_output:
            clear_output()

            selected_sources = list(energy_selector.value)
            lat_filter = df['y'] > 38 if north_toggle.value else pd.Series([True] * len(df))
            filtered_df = df[df['PriEnergySource'].isin(selected_sources) & lat_filter]

            chart = alt.Chart(filtered_df).mark_circle(size=70).encode(
                longitude='x:Q',
                latitude='y:Q',
                tooltip=['PlantName', 'County', 'PriEnergySource'],
                color='PriEnergySource:N'
            ).properties(
                title='Filtered Power Plants on Map',
                width=600,
                height=500
            ).project('albersUsa')

            display(chart)

        with count_output:
            clear_output()
            display(widgets.HTML(f"<b>Plant Count:</b> {len(filtered_df)}"))

    energy_selector.observe(update_chart, names='value')
    north_toggle.observe(update_chart, names='value')
    update_chart()

    # Global response widget
    hydro_input = widgets.Text(
        description='Answer:',
        placeholder='e.g. 12 hydro plants found across 5 counties'
    )

    submit_button = widgets.Button(description="Submit Answer")

    def on_submit(b):
        elapsed = time.time() - start_time
        selected_sources = list(energy_selector.value)
        lat_filter = df['y'] > 38 if north_toggle.value else pd.Series([True] * len(df))
        filtered_df = df[df['PriEnergySource'].isin(selected_sources) & lat_filter]

        responses.append({
            'question': 'Q: Locate hydro plants using filters and lat toggle',
            'user_answer': hydro_input.value,  # ⬅️ Use global var here
            'sources_selected': selected_sources,
            'northern_ca_only': north_toggle.value,
            'plant_count': len(filtered_df),
            'time_sec': round(elapsed, 2)
        })

        with submission_output:
            clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded.</b> Time taken: {elapsed:.2f} seconds<br>"
                #f"<b>Plants shown on map:</b> {len(filtered_df)}<br>"
                f"You can now proceed to the next task."
            ))

    submit_button.on_click(on_submit)

    display(widgets.VBox([
        energy_selector,
        north_toggle,
        hint,
        count_output,
        chart_output,
        hydro_input,  # ⬅️ Display global widget
        submit_button,
        submission_output
    ]))


# Run the task
start_hydro_filter_task()




<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 6) Which operator manages the most power plant facilities? </h2>
</div>

In [63]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
import random
import time
from IPython.display import display, clear_output

def start_operator_question():
    global answer_input
    clear_output()
    print("🏢 Task: Which operator manages the most power plants in California?")

    # Ensure responses exists
    if 'responses' not in globals():
        global responses
        responses = []

    start_time = time.time()

    # Clean and prepare top operator data
    operator_clean = df[~df['Operator Company'].str.upper().isin(['UNKNOWN', 'NOT AVAILABLE'])]
    top_ops = operator_clean['Operator Company'].value_counts().head(5).reset_index()
    top_ops.columns = ['Operator', 'Plant Count']

    # Chart (unchanged)
    chart = alt.Chart(top_ops).mark_bar().encode(
        x='Plant Count:Q',
        y=alt.Y('Operator:N', sort='-x'),
        tooltip=['Operator', 'Plant Count']
    ).properties(
        title='Top 5 Operators by Plant Count',
        width=500,
        height=300
    )
    display(chart)

    # Input + Submit
    answer_input = widgets.Text(
        placeholder='e.g. Calpine',
        description='Answer:',
        style={'description_width': 'initial'}
    )

    submit_button = widgets.Button(description="Submit Answer")
    output = widgets.Output()

    def on_submit(b):
        elapsed = time.time() - start_time
        user_val = answer_input.value

        print("DEBUG: Submitting value:", user_val)

        responses.append({
        'question': 'Q: Which operator manages the most power plant facilities?',
        'user_answer': user_val,
        'correct_answer': top_ops.iloc[0]['Operator'],
        'time_sec': round(elapsed, 2)
        })

        print("DEBUG: Total responses stored:", len(responses))

        with output:
            clear_output()
            display(widgets.HTML(
            f"<b>✅ Answer recorded:</b> {user_val}<br>"
            ))


    submit_button.on_click(on_submit)
    display(widgets.VBox([answer_input, submit_button, output]))


# 🚀 Launch the task
start_operator_question()


🏢 Task: Which operator manages the most power plants in California?


alt.Chart(...)

DEBUG: Submitting value: SCE
DEBUG: Total responses stored: 12


In [34]:
responses[-1]['user_answer']

'SCE'

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 7) What is the most common energy source for power plants in California?
</h2>
</div>

In [38]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
import random
import time
from IPython.display import display, clear_output

def start_energy_source_question():
    global source_input  # ✅ globally accessible widget
    clear_output()
    print("⚡️ Task: What is the most common energy source for power plants in California?")

    # Ensure shared response log exists
    if 'responses' not in globals():
        global responses
        responses = []

    start_time = time.time()

    # Count by energy source
    energy_counts = df['PriEnergySource'].value_counts().reset_index()
    energy_counts.columns = ['EnergySource', 'PlantCount']

    # Correct answer
    correct_source = energy_counts.iloc[0]['EnergySource']

    # Chart (top 10 energy sources)
    chart_data = energy_counts.head(10)
    chart = alt.Chart(chart_data).mark_bar().encode(
        x=alt.X('PlantCount:Q', title='Number of Plants'),
        y=alt.Y('EnergySource:N', sort='-x'),
        tooltip=['EnergySource', 'PlantCount'],
        color=alt.Color('EnergySource:N', legend=None)
    ).properties(
        title='Top 10 Energy Sources for Power Plants in California',
        height=400
    )

    # Generate choices (including correct answer)
    distractors = energy_counts[energy_counts['EnergySource'] != correct_source]['EnergySource'].tolist()
    choices = random.sample(distractors, 4) + [correct_source]
    random.shuffle(choices)

    # Question + widget
    instructions = widgets.HTML("<h4>⚡️ What is the most common energy source for power plants in California?</h4>")
    source_input = widgets.RadioButtons(
        options=choices,
        description='Your Answer:',
        layout=widgets.Layout(width='500px')
    )

    submit_button = widgets.Button(description="Submit Answer")
    submission_output = widgets.Output()

    def on_submit(b):
        elapsed = time.time() - start_time
        selected = source_input.value

        responses.append({
            'question': 'Most common energy source in CA',
            'user_answer': selected,
            'correct_answer': correct_source,
            'is_correct': selected == correct_source,
            'time_sec': round(elapsed, 2)
        })

        with submission_output:
            submission_output.clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded:</b> {selected}<br>"
                f"<b>Time taken:</b> {round(elapsed, 2)} seconds.<br>"
                f"You may proceed to the next task."
            ))

    submit_button.on_click(on_submit)

    display(chart)
    display(widgets.VBox([
        instructions,
        source_input,
        submit_button,
        submission_output
    ]))


# 🚀 Launch the task
start_energy_source_question()


⚡️ Task: What is the most common energy source for power plants in California?


alt.Chart(...)

<div style="background-color:#007acc; height:40px; margin-top:30px; margin-bottom:10px; border-radius:5px;">
  <h2 style="color:white; padding:8px 15px; margin:0;">📊 8) The visualizations helped me better understand California's energy system.
</h2>
</div>

In [58]:
import ipywidgets as widgets
import time
from IPython.display import display, clear_output

def start_visualization_opinion_question():
    global likert_scale  # ✅ Make the widget accessible globally
    clear_output()

    # Ensure global responses list exists
    if 'responses' not in globals():
        global responses
        responses = []

    start_time = time.time()

    # Widgets (unchanged)
    instructions = widgets.HTML("<h4>💬 The visualizations helped me better understand California's energy system.</h4>")
    
    likert_scale = widgets.RadioButtons(
        options=[
            "1 - Strongly Disagree",
            "2 - Disagree",
            "3 - Neutral",
            "4 - Agree",
            "5 - Strongly Agree"
        ],
        description='Your Answer:',
        layout=widgets.Layout(width='500px')
    )

    submit_button = widgets.Button(description="Submit Answer")
    submission_output = widgets.Output()

    def on_submit(b):
        elapsed = time.time() - start_time
        user_response = likert_scale.value

        responses.append({
            'question': 'Visualization helpfulness',
            'user_answer': user_response,
            'time_sec': round(elapsed, 2)
        })

        with submission_output:
            submission_output.clear_output()
            display(widgets.HTML(
                f"<b>✅ Answer recorded.</b><br>Time taken: {round(elapsed, 2)} seconds."
            ))

    submit_button.on_click(on_submit)

    display(widgets.VBox([
        instructions,
        likert_scale,
        submit_button,
        submission_output
    ]))

# 🚀 Launch the standalone question
start_visualization_opinion_question()


## CHECKS

In [93]:
#CHECK Before Submitting to Google

print("Submitting the following values:")
print("Prolific ID:", prolific_id_input.value)
print("Q1:", plant_question.value)
print("Q2:", capacity_question.value)
print("Q3:", solar_input.value)
print("Q4:", percentage_input.value)
print("Q5:", difference_input.value)
print("Q6:", hydro_input.value)
print("Q7:", answer_input.value)
print("Q8:", source_input.value)
print("Q9:", likert_scale.value)

print("Answer Input (operator):", answer_input.value)
print("User answer from responses:", responses[-1]['user_answer'])

Submitting the following values:
Prolific ID: dahsd
Q1: Hoover Dam (NV)
Q2: 5664
Q3: 2
Q4: 5.0
Q5: 150.0
Q6: 22
Q7: SCE
Q8: SUN
Q9: 3 - Neutral
Answer Input (operator): SCE
User answer from responses: SCE


In [98]:
import requests
import time
from IPython.display import clear_output
import ipywidgets as widgets

# ✅ Replace this with your actual Zapier Webhook URL
ZAPIER_WEBHOOK_URL = "https://hooks.zapier.com/hooks/catch/22581158/2x5ez87/"

# ✅ 1. Define the Zapier submission function
def send_responses_to_zapier():
    payload = {
        "prolific_id": prolific_id_input.value,
        "plant_name": plant_question.value,
        "generation_capacity": capacity_question.value,
        "solar_count": solar_input.value,
        "percent_in_la": percentage_input.value,
        "mw_difference": difference_input.value,
        "hydro_count": hydro_input.value,
        "operator": answer_input.value,
        "energy_source": source_input.value,
        "visual_opinion": likert_scale.value
    }

    try:
        response = requests.post(ZAPIER_WEBHOOK_URL, json=payload)
        return response.status_code
    except Exception as e:
        print(f"❌ Error sending to Zapier: {e}")
        return None

# ✅ 2. Validation helper
def validate_inputs():
    missing = []
    if not prolific_id_input.value.strip(): missing.append("Prolific ID")
    if not plant_question.value: missing.append("Q1 - Plant Name")
    if not capacity_question.value: missing.append("Q2 - Capacity")
    if not solar_input.value: missing.append("Q3 - Solar Count")
    if not percentage_input.value: missing.append("Q4 - % in LA")
    if not difference_input.value: missing.append("Q5 - MW Difference")
    if not hydro_input.value: missing.append("Q6 - Hydro")
    if not answer_input.value: missing.append("Q7 - Operator")
    if not source_input.value: missing.append("Q8 - Energy Source")
    if not likert_scale.value: missing.append("Q9 - Visualization Opinion")

    if missing:
        print("⚠️ Please complete the following fields before submitting:")
        for field in missing:
            print(f"• {field}")
        return False
    return True

# ✅ 3. Submit button logic
submit_button = widgets.Button(description="📩 Submit Survey", button_style='success')
submission_output = widgets.Output()
start_time = time.time()

def on_survey_submit(b):
    with submission_output:
        clear_output()
        print("📤 Sending your survey responses...")

        if not validate_inputs():
            print("❌ Submission cancelled. Please complete all required fields.")
            return

        status = send_responses_to_zapier()

        if status == 200:
            print("✅ Submission successful! Thank you for completing the survey.")
            submit_button.disabled = True
        elif status is None:
            print("❌ An error occurred while sending the survey.")
        else:
            print(f"❌ Submission failed. Status code: {status}")

        end_time = time.time()
        print(f"⏱️ Time taken: {round(end_time - start_time, 2)} seconds")

# ✅ 4. Attach click event and display the button
submit_button.on_click(on_survey_submit)
display(submit_button, submission_output)


Button(button_style='success', description='📩 Submit Survey', style=ButtonStyle())

Output()